In [1]:
from nbgwas import NBGWAS_snp2gene as snp2gene
import os
import pandas as pd
import time

In [28]:
# Generate cluster file to run snp2gene assignments on cluster
f = open('/cellar/users/jkhuang/Data/nosnap/Projects/NBGWAS/snp2gene/param_file_2018-03-21.txt', 'w')

hg19_gene_pos_file = '/cellar/users/jkhuang/Data/Projects/Network_GWAS/Data/IGAP/SNP_Gene_Locations/glist-hg19_fmt.txt'
windows = [5, 10, 50, 100, 500, 1000]
WTCCC_data_dir = '/cellar/users/jkhuang/Data/Projects/Network_GWAS/Data/WTCCC/WTCCC_SNPs/'
fn_cols = '0,1,2,6'
for fn in os.listdir(WTCCC_data_dir):
    if fn.endswith('_WTCCC_snptest_galaxy-LiftOver-hg19_summary_table.csv'):
        disease = fn.split('_')[0]
        job_name = disease+'_WTCCC'
        for size in windows:
            f.write(' '.join([hg19_gene_pos_file, WTCCC_data_dir, fn, fn_cols, job_name, str(size)])+'\n')
        
IGAP_data_dir = '/cellar/users/jkhuang/Data/Projects/Network_GWAS/Data/IGAP/IGAP_SNPs/'
fn = 'IGAP_stage_1.txt'
fn_cols = '2,0,1,7'
for size in windows:
    job_name = 'ALZ_IGAP'
    f.write(' '.join([hg19_gene_pos_file, IGAP_data_dir, fn, fn_cols, job_name, str(size)])+'\n')
    
f.close()


In [ ]:
# Load hg19 gene locations (gene locations from VEGAS2 gencode file)
hg19_gene_pos_file = '/cellar/users/jkhuang/Data/Projects/Network_GWAS/Data/IGAP/SNP_Gene_Locations/glist-hg19_fmt.txt'
hg19_gene_pos = snp2gene.load_gene_pos(hg19_gene_pos_file, delimiter='\t', header=True, cols='0,1,2,3')

In [ ]:
# Assign SNP p-values to genes for WTCCC data over varying window sizes using min-p and closest-gene
windows = [5, 10, 50, 100, 500, 1000]
WTCCC_data_dir = '/cellar/users/jkhuang/Data/Projects/Network_GWAS/Data/WTCCC/WTCCC_SNPs/'
for fn in os.listdir(WTCCC_data_dir):
    disease = fn.split('_')[0]
    print 'Disease:', disease
    WTCCC_snp_table = snp2gene.load_SNP_pvals(WTCCC_data_dir+fn, delimiter='\t', header=True, cols='0,1,2,6')
    for size in windows:
        print 'Window size:', size, 'kb'
        
        starttime1 = time.time()
        closest_gene_table = snp2gene.closest_gene(WTCCC_snp_table, hg19_gene_pos, size)
        closest_gene_table.to_csv(WTCCC_data_dir+disease+'_WTCCC_closest_gene_'+repr(window)+'kb.csv', sep='\t') 
        print 'Closest gene p-values assigned:', time.time()-starttime1
        
        starttime2 = time.time()
        min_p_table = snp2gene.min_p(WTCCC_snp_table, hg19_gene_pos, size)
        min_p_table.to_csv(WTCCC_data_dir+disease+'_WTCCC_min_p_'+repr(window)+'kb.csv', sep='\t') 
        print 'Minimum p-values assigned:', time.time()-starttime2
        

In [ ]:
# Assign SNP p-values to genes for WTCCC data over varying window sizes using min-p and closest-gene
IGAP_snp_file = '/cellar/users/jkhuang/Data/Projects/Network_GWAS/Data/IGAP/IGAP_SNPs/IGAP_stage_1.txt'
IGAP_snp_table = snp2gene.load_SNP_pvals(WTCCC_data_dir+fn, delimiter='\t', header=True, cols='2,0,1,7')
for size in windows:
    print 'Window size:', size, 'kb'

    starttime1 = time.time()
    closest_gene_table = snp2gene.closest_gene(IGAP_snp_table, hg19_gene_pos, size)
    closest_gene_table.to_csv(WTCCC_data_dir+'ALZ_IGAP_closest_gene_'+repr(window)+'kb.csv', sep='\t') 
    print 'Closest gene p-values assigned:', time.time()-starttime1

    starttime2 = time.time()
    min_p_table = snp2gene.min_p(IGAP_snp_table, hg19_gene_pos, size)
    min_p_table.to_csv(WTCCC_data_dir+'ALZ_IGAP_min_p_'+repr(window)+'kb.csv', sep='\t') 
    print 'Minimum p-values assigned:', time.time()-starttime2